**Project Proposal:** Predicting the Seeding of a Tennis Winner Through Data Analysis

**Predictive Question:**
Given a winner's number of aces (w_ace) and number of serve points (w_pvpt), predict what seed number (1-3) this player is, in a clay surface tournament.

**Introduction:** The dateset we employed for this project included information for every ATP (Association of Tennis Professionals) match dating back to 1968.  However, for this project, we are focusing on data from the years 2017 to 2019. The central question of this project is to determine the seed number for players on clay surface tournaments based on two factors: the number of aces achieved by the winner (w_ace) and the number of serve points (w_pvpt).
A "seed" is an initial ranking in a tournament to ensure top players don't face each other early, An "ace" is an untouched serve by the receiver, winning a point for the server. "Serve points" are serves until the receiver fails to return. The "surface" is the court material (hard, grass, clay, or carpet), impacting playing style and match predictions.

**Preliminary data analysis:**

In [1]:
install.packages("kknn")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
# Load the required libraries
library(tidyverse)
library(repr)
library(tidymodels)
library(kknn)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

In [ ]:
# Read the data into R
url <- "https://raw.githubusercontent.com/Caseyxyq/Dsci100_group44/main/atp2017-2019.csv"
atp_data <- read_csv(url)
head(atp_data, n = 5)

# Change the winner_seed column type to <dbl> from <chr>
# Filter for Clay type of surface and winner seed values less than equal to 3
filtered_selected_columns <- atp_data |>
    mutate(winner_seed = as.numeric(winner_seed)) |>
    filter(surface == "Clay", winner_seed <= 3.00) |> 
    select(surface, w_ace, w_svpt, winner_seed)
filtered_selected_columns

# Remove the NA values 
final_data <- filtered_selected_columns |>
    drop_na()
head(final_data, n = 5)

# Summarize final data by reporting the number of observations in each winner seed
number_of_obs <- group_by(final_data, winner_seed) |>
    summarize(count = n())
number_of_obs

# Summarize final data by reporting means of the two predictors
means_of_predictors <- final_data |>
    select(w_ace, w_svpt) |>
    summarize(mean_w_ace = mean(w_ace),
              mean_w_svpt = mean(w_svpt))
means_of_predictors

# Summarize final data by reporting maximum values of the two predictors
max_of_predictors <- final_data |>
    select(w_ace, w_svpt) |>
    summarize(max_w_ace = max(w_ace),
              max_w_svpt = max(w_svpt))
max_of_predictors

# Split the data into training and testing data 
set.seed(3456) 

atp_split <- initial_split(final_data, prop = 0.75, strata = winner_seed)
atp_train <- training(atp_split)
atp_test <- testing(atp_split)


atp_recipe <- recipe(w_svpt~ w_ace, data = atp_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())
atp_recipe

atp_spec <- nearest_neighbor(weight_func = "rectangular", 
                              neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("regression")
atp_spec

atp_vfold <- vfold_cv(atp_train, v = 5, strata = winner_seed)

atp_wkflw <- workflow() |>
  add_recipe(atp_recipe) |>
  add_model(atp_spec)
atp_wkflw

gridvals <- tibble(neighbors = seq(from = 1, to = 200, by = 3))

atp_results <- atp_wkflw |>
  tune_grid(resamples = atp_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "rmse")
atp_results

# Plot the data in a scatterplot

options(repr.plot.width = 10, repr.plot.height = 12)


atp_plot <- atp_train |>  
  ggplot(aes(x = w_ace, 
               y = w_svpt, 
               colour = as.factor(winner_seed))) +
    geom_point() +
    labs(x = "Winner Ace",
         y = "Winner Serve Point",
         color = 'Winner Seed') +
    ggtitle("Predicting the tournament seeding of a tennis winner") +
    theme(text = element_text(size = 20))


New names:
• `` -> `...1`
Rows: 6866 Columns: 50
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (16): tourney_id, tourney_name, surface, tourney_level, winner_seed, win...
dbl (34): ...1, draw_size, tourney_date, match_num, winner_id, winner_ht, wi...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,⋯,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2,⋯,54,34,20,14,10,15,9,3590,16,1977
1,2019-M020,Brisbane,Hard,32,A,20181231,299,106421,4,⋯,52,36,7,10,10,13,16,1977,239,200
2,2019-M020,Brisbane,Hard,32,A,20181231,298,105453,2,⋯,27,15,6,8,1,5,9,3590,40,1050
3,2019-M020,Brisbane,Hard,32,A,20181231,297,104542,NA,⋯,60,38,9,11,4,6,239,200,31,1298
4,2019-M020,Brisbane,Hard,32,A,20181231,296,106421,4,⋯,56,46,19,15,2,4,16,1977,18,1855


Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `winner_seed = as.numeric(winner_seed)`.
Caused by warning:
! NAs introduced by coercion”


surface,w_ace,w_svpt,winner_seed
<chr>,<dbl>,<dbl>,<dbl>
Clay,3,94,3
Clay,4,40,3
Clay,4,52,3
Clay,4,89,1
Clay,3,71,3
Clay,0,62,1
Clay,2,86,3
Clay,8,65,3
Clay,5,76,3


surface,w_ace,w_svpt,winner_seed
<chr>,<dbl>,<dbl>,<dbl>
Clay,3,94,3
Clay,4,40,3
Clay,4,52,3
Clay,4,89,1
Clay,3,71,3


...

max_w_ace,max_w_svpt
<dbl>,<dbl>
25,173


Recipe

Inputs:

      role #variables
   outcome          1
 predictor          1

Operations:

Scaling for all_predictors()
Centering for all_predictors()

K-Nearest Neighbor Model Specification (regression)

Main Arguments:
  neighbors = tune()
  weight_func = rectangular

Computational engine: kknn 


══ Workflow ════════════════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: nearest_neighbor()

── Preprocessor ────────────────────────────────────────────────────────────────
2 Recipe Steps

• step_scale()
• step_center()

── Model ───────────────────────────────────────────────────────────────────────
K-Nearest Neighbor Model Specification (regression)

Main Arguments:
  neighbors = tune()
  weight_func = rectangular

Computational engine: kknn 


! Fold1: preprocessor 1/1, model 1/1: 199 samples were requested but there were 161 rows in the data. 156 will...



**Method:** We'll predict seed number of a winner using two numerical variables: winner's number of aces and winner's number of serve points.
Firstly, we read the dataset from a URL so it can be seen without downloading locally. To clean and wrangle, we select columns that will be used in our prediction: surface, w_ace, w_pvpt and winner_seed. As we predict the top 3 seed numbers, we filter for seed numbers (=<3), filter clay as the type of surface, and remove all NA values. With our final dataset, we summarize values into useful tables. Next, we split the data into training set and test set so the classifier is built based on the training set and later used to predict data in the test set. Finally, we visualize the data by creating a scatter plot with winner's number of aces and winner's number of serve points on each axis, and different colored seed numbers for differentiation. Labels of axes, legend, and plot title are also added to make effective visualizations.

**Outcomes and Significance:** With our analysis, we’ll have a clear visualization demonstrating the relationship between the seed number of a tennis player, aces achieved by the winner, and serve points received by the winner on clay ground. We can classify the “overall seed” of winners who obtain a certain number of aces and serve points. Our data can verify the accuracy of seeds (how accurately does seed reflect player skill) based on whether a higher seed obtains more aces and serve points. If it reflects that winner seed assignment is inaccurate, it could lead to innovation in seed prediction. Since higher seed players are better, we can also draw explanations to questions like: “ If better players (higher seed) generally get more aces/serve points.” Our set of data can also be compared to a similar set where data is filtered to a different surface, and that can be taken to see if surfaces affect the overall study of the example questions above. Overall, we hope the results will draw more insight for future questions regarding relationships between winner seed and winner statistics. 
